In [1]:
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import cv2
import os,sys
if 'google.colab' in sys.modules:
    from google.colab import files

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'2.7.0'

## Tensorflow Generative Adversarial Network

### DCGAN

Radford, Metz & Chintala (2016) Unsupervised Representation Learning With Deep Convolutional Generative Adversarial Networks. 

https://arxiv.org/pdf/1511.06434.pdf

In [3]:
def build_generator(dim, depth, channels=1, 
                    inputDim=100,
                    outputDim=512):
    model = Sequential()
    # Two Fully Connected layers with Batch Normalization
    model.add(Dense(input_dim=inputDim, units=outputDim))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
  
    model.add(Dense(dim * dim * depth))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    
    # Reshape the output of the previous layer, upsample
    # apply a transposed convolution (i.e. deconvolution), RELU, and BN
    model.add(Reshape((dim, dim, depth))) #Input shape
    model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
                              padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    # Apply another upsample and transposed convolution, but
    # with TANH activation
    model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
                              padding="same"))
    model.add(Activation("tanh"))

    return model
    

In [4]:
def build_discriminator(width, height, depth, alpha=0.2):
    model = Sequential()
    # Two conv2d layers
    model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
                     input_shape=(height, width, depth)))
    model.add(LeakyReLU(alpha=alpha))
    model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
    model.add(LeakyReLU(alpha=alpha))
    
    # Flatten Layer
    model.add(Flatten())
    
    #  One linear hidden layer with leaky relu
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=alpha))
    # output layer with sigmoid activation
    model.add(Dense(1))
    model.add(Activation("sigmoid"))

    return model

#### Load the Fashion MNIST dataset
* Stack the training and testing data points so we have additional training data

In [5]:
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])
trainImages.shape

(70000, 28, 28)

#### Preprocess

Add in an extra dimension for the channel and scale the images
into the range [-1, 1] (which is the range of the tanh function)

In [6]:
trainImages = np.expand_dims(trainImages, axis=-1)
trainImages = (trainImages.astype("float") - 127.5) / 127.5
trainImages.shape

(70000, 28, 28, 1)

#### Build models and compile

In [7]:
lr = 0.0002
num_epochs = 10

G_model = build_generator(7, 64, channels=1)
D_model = build_discriminator(28, 28, 1)
D_model.compile(loss="binary_crossentropy", 
                optimizer=Adam(lr=lr,beta_1=0.5,decay=lr/num_epochs))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [8]:
G_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               51712     
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 3136)              1608768   
                                                                 
 activation_1 (Activation)   (None, 3136)              0         
                                                                 
 batch_normalization_1 (Batc  (None, 3136)             12544     
 hNormalization)                                        

In [9]:
D_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        832       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          51264     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1606144   
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 512)              

#### Build the GAN model
* Set the discriminator to **not** be trainable, i.e. freeze the weights
* Compose  discriminator with the generator

In [10]:
D_model.trainable = False
ganInput = Input(shape=(100,))
ganOutput = D_model(G_model(ganInput))
GAN_model = Model(ganInput, ganOutput)

In [11]:
GAN_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         1727233   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1658753   
                                                                 
Total params: 3,385,986
Trainable params: 1,719,873
Non-trainable params: 1,666,113
_________________________________________________________________


In [12]:

GAN_model.compile(
            loss="binary_crossentropy", 
            optimizer=Adam(lr=lr, beta_1=0.5, decay=lr/num_epochs))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


#### Train the GAN

In [13]:
def prepare_images(G,imageBatch, batch_size):
    ''' Select the next batch of images, then randomly generate
        noise for the generator to predict on.
        Construct class labels for the discriminator, and shuffle
        the data'''
    noise = np.random.uniform(-1, 1, size=(batch_size, 100))
    # Generate images using the noise + generator model
    genImages = G.predict(noise, verbose=0)
    X = np.concatenate((imageBatch, genImages))#Concat real and generated
    y = ([1] * batch_size) + ([0] * batch_size)
    y = np.reshape(y, (-1,))
    return shuffle(X, y)

def GAN_output(G,epoch,noise,output):
    '''
    Visualize the output of the generator model on our benchmark data
    Make predictions on the benchmark noise, scale it back
    to the range [0, 255], and generate the montage'''
    fp = [output, f"epoch_{epoch + 1:03}_output.png"]   
    images = G.predict(noise)
    images = ((images * 127.5) + 127.5).astype("uint8")
    images = np.repeat(images, 3, axis=-1)
    vis = build_montages(images, (28, 28), (16, 16))[0]
    # write the visualization to disk
    fp = os.path.sep.join(fp)
    res = cv2.imwrite(fp, vis)
    print(fp,' ',res)

In [14]:
def train_gan(G,D,GAN,trainImages,num_epochs,outDir):
    benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))
    batch_size = 128
    batchesPerEpoch = int(trainImages.shape[0]/batch_size)
    for epoch in range(0, num_epochs):
        for i in range(0, batchesPerEpoch):
            imageBatch = trainImages[i*batch_size:(i+1)*batch_size]
            X,y = prepare_images(G,imageBatch,batch_size)
            # Train the discriminator on the data
            D_Loss = D.train_on_batch(X, y)

            # Train the generator via the adversarial model by
            # (1) generating random noise and 
            # (2) training generator with the discriminator weights frozen
            noise = np.random.uniform(-1, 1, (batch_size, 100))
            fakeLabels = [1] * batch_size
            fakeLabels = np.reshape(fakeLabels, (-1,))
            GAN_Loss = GAN.train_on_batch(noise, fakeLabels)
            if not i%150:
                print(
                f"Epoch {epoch+1}_{i}: D loss={D_Loss:.4f}, GAN loss={GAN_Loss:.4f}")
          
            # Write predicted images to disk
            if outDir and (i == batchesPerEpoch - 1): # End of epoch
                GAN_output(G,epoch,benchmarkNoise,outDir)
        

In [15]:


output = "Images"
train_gan(G_model,D_model,GAN_model,
          trainImages,
          num_epochs,
          output)

Epoch 1_0: D loss=0.6497, GAN loss=0.5647
Epoch 1_150: D loss=0.0003, GAN loss=0.0003
Epoch 1_300: D loss=0.4648, GAN loss=1.0464
Epoch 1_450: D loss=0.5234, GAN loss=0.7322
Images/epoch_001_output.png   False
Epoch 2_0: D loss=0.5726, GAN loss=1.3877
Epoch 2_150: D loss=0.5650, GAN loss=0.8348
Epoch 2_300: D loss=0.5625, GAN loss=0.7761
Epoch 2_450: D loss=0.6495, GAN loss=1.5296
Images/epoch_002_output.png   False
Epoch 3_0: D loss=0.5976, GAN loss=0.7798
Epoch 3_150: D loss=0.6183, GAN loss=0.9943
Epoch 3_300: D loss=0.6018, GAN loss=1.1032
Epoch 3_450: D loss=0.6265, GAN loss=1.1012
Images/epoch_003_output.png   False
Epoch 4_0: D loss=0.6254, GAN loss=1.0488
Epoch 4_150: D loss=0.6521, GAN loss=0.7308
Epoch 4_300: D loss=0.6321, GAN loss=0.9932
Epoch 4_450: D loss=0.6563, GAN loss=0.5920
Images/epoch_004_output.png   False
Epoch 5_0: D loss=0.6467, GAN loss=0.9133
Epoch 5_150: D loss=0.6391, GAN loss=0.9493
Epoch 5_300: D loss=0.6079, GAN loss=0.7393
Epoch 5_450: D loss=0.6348, GA

### References


https://www.pyimagesearch.com/2020/11/16/gans-with-keras-and-tensorflow/